In [1]:
import pandas as pd
import numpy as np
import matlab.engine
import os

In [2]:
# Setup MATLAB Engine
print("Starting MATLAB Engine...")
eng = matlab.engine.start_matlab()
# Add current directory to path so it finds 'ConvexSpline.m'
eng.addpath(os.getcwd(), nargout=0)

Starting MATLAB Engine...


In [3]:
# Function that feeds our dataframe to the Matlab script
def fit_group_convex_spline(g: pd.DataFrame) -> pd.Series:
    """
    Fits spline in MATLAB and evaluates it immediately on the group's data.
    """
    # Filter for valid fitting data
    valid_data = g[['strike_price', 'bs_put_price', 'S0']].dropna()
    
    # Safety check: need enough points to fit (e.g. at least 4)
    if len(valid_data) < 4:
        return pd.Series(np.nan, index=g.index, name='bs_put_price_fit')

    # Prepare Inputs for MATLAB
    x_fit = matlab.double(valid_data['strike_price'].tolist())
    y_fit = matlab.double(valid_data['bs_put_price'].tolist())
    spot  = float(valid_data['S0'].iloc[0])
    
    # Empty placeholder for optional args (sOrder, numPieces, MinConstraint)
    empty = matlab.double([])

    try:
        # Call ConvexSpline.m
        sp_struct = eng.ConvexSpline(x_fit, y_fit, spot, empty, empty, empty, 0.1, nargout=1)

        # Evaluate the spline on all strikes in the group
        x_eval = matlab.double(g['strike_price'].tolist())
        
        # Calculate the fitted values using fnval
        fitted_vals = eng.fnval(sp_struct, x_eval)
        
        # Convert result back to 1D numpy array
        result = np.array(fitted_vals).flatten()
        
    except Exception as e:
        print(f"Fit failed: {e}")
        result = np.full(len(g), np.nan)

    return pd.Series(result, index=g.index, name='bs_put_price_fit')

In [4]:
# Import the test data
test_data = pd.read_csv('test_data_matlab_output.csv')
test_data['S0'] = 4707.09
test_data = test_data.rename(columns={'Unnamed: 2': 'Matlab Results'})
test_data.head()

,strike_price,bs_put_price,Matlab Results,S0
0,3750.0,0.005878,0.001827,4707.09
1,3875.0,0.007928,0.006164,4707.09
2,3910.0,0.009116,0.007387,4707.09
3,3925.0,0.008903,0.007912,4707.09
4,3940.0,0.009498,0.008438,4707.09


In [5]:
test_data['Python Results'] = fit_group_convex_spline(test_data[['strike_price', 'bs_put_price', 'S0']])

In [6]:
test_data

,strike_price,bs_put_price,Matlab Results,S0,Python Results
0,3750.0,0.005878,0.001827,4707.09,0.001827
1,3875.0,0.007928,0.006164,4707.09,0.006164
2,3910.0,0.009116,0.007387,4707.09,0.007387
3,3925.0,0.008903,0.007912,4707.09,0.007912
4,3940.0,0.009498,0.008438,4707.09,0.008438
...,...,...,...,...,...
155,5000.0,286.617180,286.582718,4707.09,286.582718
156,5200.0,486.205216,486.222798,4707.09,486.222798
157,5400.0,685.931362,685.926237,4707.09,685.926237
158,5600.0,885.660988,885.629858,4707.09,885.629858
